# Asset Hierarchy Migration

**Prerequisite**:

- Access to a CDF Project.
- Know how to install and setup Python.
- Launch a Python notebook.



In this tutorial, we will show you how to migrate an asset hierarchy to a data model representing the same hierarchy in CDF.

This tutorial is also a good demonstration of the width of capabilities for neat, going from extracting data,
creating a data model, export the data model, and load the data.

## Extract Data from Asset Hierarchy

We will start by extracting the data from an existing asset hierarchy. 

The example we will use in this tutorial is an asset hierarchy for pumps, shown below in CDF's classic Data Exploration

<img src="../artifacts/figs/asset_hierarchy_lift_pump_stations.png" width="400">

In [2]:
from cognite.neat import get_cognite_client
from cognite.neat.graph import extractors, NeatGraphStore

We start by importing the neat extractors, NeatGraphStore, and a utility method for getting a neat client.

In [3]:
store = NeatGraphStore.from_memory_store()

The NeatGraphStore is a triple store that can store both data as well as schemas/data-models.

This store we can populate by extracting from one of the supported sources for neat. 

To see which extractors are available we print the `extractors` module.

In [4]:
extractors

,Extractor,Description
0,AssetsExtractor,Extract data from Cognite Data Fusions Assets ...
1,MockGraphGenerator,Class used to generate mock graph data for pur...
2,RelationshipsExtractor,Extract data from Cognite Data Fusions Relatio...
3,TimeSeriesExtractor,Extract data from Cognite Data Fusions TimeSer...
4,SequencesExtractor,Extract data from Cognite Data Fusions Sequenc...
5,EventsExtractor,Extract data from Cognite Data Fusions Events ...
6,FilesExtractor,Extract data from Cognite Data Fusions files m...
7,LabelsExtractor,Extract data from Cognite Data Fusions Labels ...
8,RdfFileExtractor,Extract data from RDF files into Neat.
9,DexpiExtractor,DEXPI-XML extractor of RDF triples


At the top of the list we see the AssetsExtractor which is what we are looking for

In [5]:
extractors.AssetsExtractor

cognite.neat.graph.extractors._classic_cdf._assets.AssetsExtractor

We see that there is a `.from_hierarchy` fatory method that fits well with what we want to do

In [6]:
client = get_cognite_client()

Found .env file in repository root. Loaded variables from .env file.


Note that the utility function `get_cognite_client` will prompt us for credentials if it doesn't find a `.env` file in the repo root.

In [7]:
asset_extractor = extractors.AssetsExtractor.from_hierarchy(client, root_asset_external_id="lift_pump_stations:root")

The asset extractor is now ready and can be written to the store.

In [8]:
store.write(asset_extractor)

We can inspect the store to see what changes have been applied to it:

In [9]:
store

,Agent,Activity,Entity,Description
0,http://purl.org/cognite/neat#agent,http://purl.org/cognite/neat#activity-1678c7c5...,http://purl.org/cognite/neat#graph-store,Initialize graph store as Memory
1,http://purl.org/cognite/neat#agent,http://purl.org/cognite/neat#activity-1678c7c5...,http://purl.org/cognite/neat#graph-store,Extracted triples to graph store using AssetsE...


## Infering Data Model from Data

To infer a data model from the data we will use an importer

In [10]:
from cognite.neat.rules import importers

In [11]:
importers

,Importer,Description
0,OWLImporter,Convert OWL ontology to tables/ transformation...
1,DMSImporter,Imports a Data Model from Cognite Data Fusion.
2,ExcelImporter,Import rules from an Excel file.
3,GoogleSheetImporter,Import rules from a Google Sheet.
4,DTDLImporter,Importer from Azure Digital Twin - DTDL (Digit...
5,YAMLImporter,Imports the rules from a YAML file.
6,InferenceImporter,Infers rules from a triple store.


We see that there are multiple importers availabe, but we will use the `InferenceImporter`

In [12]:
importers.InferenceImporter

cognite.neat.rules.importers._inference2rules.InferenceImporter

In [13]:
importer = importers.InferenceImporter.from_graph_store(store)

Then we use the `.to_rules` method to convert the data into `Rules` which is `Neat`'s format for data models.

In [14]:
rules, issues = importer.to_rules()

First we check if there were any issues with creating the rules, and we fing one warning

In [15]:
issues

,field_name,value
0,property,Shape__Length


Then, we inspect the classes found along with the properties

In [16]:
rules.classes

,class_,reference,match_type,comment
0,neat:Asset,http://purl.org/cognite/neat#Asset,exact,"Inferred from knowledge graph, where this clas..."


In [17]:
rules.properties

,class_,property_,value_type,max_count,reference,transformation,comment,inherited
0,neat:Asset,name,string,1,http://purl.org/cognite/neat#name,neat:Asset(neat:name),Class <Asset> has property <name> with value t...,False
1,neat:Asset,external_id,string,1,http://purl.org/cognite/neat#external_id,neat:Asset(neat:external_id),Class <Asset> has property <external_id> with ...,False
2,neat:Asset,created_time,dateTime,1,http://purl.org/cognite/neat#created_time,neat:Asset(neat:created_time),Class <Asset> has property <created_time> with...,False
3,neat:Asset,last_updated_time,dateTime,1,http://purl.org/cognite/neat#last_updated_time,neat:Asset(neat:last_updated_time),Class <Asset> has property <last_updated_time>...,False
4,neat:Asset,DesignPointFlowGPM,double,1,http://purl.org/cognite/neat#DesignPointFlowGPM,neat:Asset(neat:DesignPointFlowGPM),Class <Asset> has property <DesignPointFlowGPM...,False
5,neat:Asset,DesignPointHeadFT,double,1,http://purl.org/cognite/neat#DesignPointHeadFT,neat:Asset(neat:DesignPointHeadFT),Class <Asset> has property <DesignPointHeadFT>...,False
6,neat:Asset,Enabled,double,1,http://purl.org/cognite/neat#Enabled,neat:Asset(neat:Enabled),Class <Asset> has property <Enabled> with valu...,False
7,neat:Asset,HighHeadShutOff,double,1,http://purl.org/cognite/neat#HighHeadShutOff,neat:Asset(neat:HighHeadShutOff),Class <Asset> has property <HighHeadShutOff> w...,False
8,neat:Asset,LowHeadFT,double,1,http://purl.org/cognite/neat#LowHeadFT,neat:Asset(neat:LowHeadFT),Class <Asset> has property <LowHeadFT> with va...,False
9,neat:Asset,LowHeadFlowGPM,double,1,http://purl.org/cognite/neat#LowHeadFlowGPM,neat:Asset(neat:LowHeadFlowGPM),Class <Asset> has property <LowHeadFlowGPM> wi...,False


We notice that for example the `PumpModel` is both an integer and a string, as the `Inference` found data of both types.

We can inspect the comment from the `Inference` type:

In [18]:
rules.properties.data[24].comment

'Class <Asset> has property <PumpModel> with value type <integer> which occurs <6> times in the graph, with value type <string> which occurs <140> times in the graph'

And we see that this is most likely a string as that occured much more for this field in the graph than the integer.

## Exporting Data Model

Lets export our newly created data model to CDF

In [18]:
from cognite.neat.rules import exporters

In [19]:
exporters

,Exporter,Description
0,DMSExporter,Export rules to Cognite Data Fusion's Data Mod...
1,SemanticDataModelExporter,Exports rules to a semantic data model.
2,OWLExporter,Exports rules to an OWL ontology.
3,SHACLExporter,Exports rules to a SHACL graph.
4,ExcelExporter,Export rules to Excel.
5,YAMLExporter,Export rules to YAML.


To export the data model we use the `DMSExporter`.

In [20]:
exporters.DMSExporter

cognite.neat.rules.exporters._rules2dms.DMSExporter

In [21]:
exporter = exporters.DMSExporter()

In [22]:
result = exporter.export_to_cdf(rules, client)

C:\Users\AndersAlbert\AppData\Local\pypoetry\Cache\virtualenvs\cognite-neat-WszCo0Uu-py3.12\Lib\site-packages\pydantic\main.py:176: MoreThanOneNonAlphanumericCharacterWarning: ('property', 'Shape__Length')
  self.__pydantic_validator__.validate_python(data, self_instance=self)


In [42]:
result

,name,created
0,spaces,1
1,containers,1
2,views,1
3,data_models,1


We see the data model was succesfully created.

## Populating Data Model

To populate the data model in CDF, we use a loader.

In [19]:
from cognite.neat.graph import loaders

In [20]:
loaders

,Loader,Description
0,DMSLoader,Load data from Cognite Data Fusions Data Model...


In [21]:
loaders.DMSLoader

cognite.neat.graph.loaders._rdf2dms.DMSLoader

To load the data from the graph store, we add the rules to the store

In [22]:
store.add_rules(rules)

In [23]:
store

,Agent,Activity,Entity,Description
0,http://purl.org/cognite/neat#agent,http://purl.org/cognite/neat#activity-1678c7c5...,http://purl.org/cognite/neat#graph-store,Initialize graph store as Memory
1,http://purl.org/cognite/neat#agent,http://purl.org/cognite/neat#activity-1678c7c5...,http://purl.org/cognite/neat#graph-store,Extracted triples to graph store using AssetsE...
2,http://purl.org/cognite/neat#agent,http://purl.org/cognite/neat#activity-1678c7c5...,http://purl.org/cognite/neat#graph-store,Added rules to graph store as InformationRules
3,http://purl.org/cognite/neat#agent,http://purl.org/cognite/neat#activity-1678c7c5...,http://purl.org/cognite/neat#graph-store,Upsert prefixes to graph store


This is necessary for the store to be ready to load data to an extrenal system.

First, we ensure the instance space exists.

In [28]:
from cognite.client import data_modeling as dm

In [50]:
client.data_modeling.spaces.apply(dm.SpaceApply("sp_pump_station"))

,value
space,sp_pump_station
is_global,False
last_updated_time,2024-06-27 14:57:20.678000
created_time,2024-06-27 14:57:20.678000


We can now use the loader to populate the data model in CDF.

In [26]:
loader = loaders.DMSLoader.from_rules(rules, store, instance_space="sp_pump_station")

In [30]:
result = loader.load_into_cdf(client)

In [31]:
result

""


In [24]:
store.queries.list_types()

[(rdflib.term.URIRef('http://purl.org/cognite/neat#Asset'),)]

In [25]:
store.read("Asset")

[('Asset_8786745865150192', 'VFDSetting', 'nan'),
 ('Asset_4582156459236482', 'external_id', 'lift_station:pheasant_run'),
 ('Asset_413299123404753', 'DesignPointHeadFT', '72.4'),
 ('Asset_285703678588887',
  'description',
  'Col Lift Sta. #77   Tumalo Heights - Tumalo Heights, in front of 2551 NW 1ST St.'),
 ('Asset_690992856868092', 'DesignPointFlowGPM', '95.0'),
 ('Asset_1328367249298685', 'LowHeadFT', 'nan'),
 ('Asset_673071391885663', 'description', ' '),
 ('Asset_2849528494272170', 'LifeCycleStatus', 'I'),
 ('Asset_6416891387482527', 'LowHeadFT', '130.0'),
 ('Asset_6996457070033380', 'HighHeadShutOff', '0.0'),
 ('Asset_4161802645953289', 'Shape__Length', 'nan'),
 ('Asset_673071391885663', 'name', 'Pump SGVF2022L'),
 ('Asset_2332136792376110', 'parent', 'Asset_2622548696369535'),
 ('Asset_285703678588887', 'DesignPointHeadFT', '66.5'),
 ('Asset_6037474207471867', 'HighHeadShutOff', 'nan'),
 ('Asset_2738034707487198', 'PumpHP', '5.0'),
 ('Asset_2849528494272170', 'DesignPointFlowG

In [28]:
dms_rules = rules.as_dms_architect_rules()

In [29]:
dms_rules.metadata

,value
role,DMS Architect
data_model_type,enterprise
schema_,partial
extension,addition
space,neat
name,Inferred Model
description,None
external_id,inferred_model
version,inferred
creator,NEAT
